<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/modeling_AlexNet/test_sample_AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git

In [2]:
%cd PyTorch-Architectures/modeling_AlexNet/

/content/PyTorch-Architectures/modeling_AlexNet


In [12]:
import time
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

from model import AlexNet

In [5]:
# SETTINGS

# Model Settings
random_seed=1
learning_rate=0.0001
batch_size=256
num_epochs=20

# Architecture
num_classes=10

# Other
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Dataset
train_indices=torch.arange(0, 48000)
valid_indices=torch.arange(48000, 50000)

train_transform=transforms.Compose([transforms.Resize((70, 70)),
                                    transforms.RandomCrop((64, 64)),
                                    transforms.ToTensor()])

test_transform=transforms.Compose([transforms.Resize((70, 70)),
                                   transforms.CenterCrop((64, 64)),
                                   transforms.ToTensor()])

train_and_valid=datasets.CIFAR10(root="data", train=True, transform=train_transform, download=True)

train_dataset=Subset(train_and_valid, train_indices)
valid_dataset=Subset(train_and_valid, valid_indices)
test_dataset=datasets.CIFAR10(root="data", train=False, transform=test_transform, download=False)

train_loader=DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=4)

valid_loader=DataLoader(dataset=valid_dataset, batch_size=batch_size, num_workers=4)

test_loader=DataLoader(dataset=test_dataset, batch_size=batch_size, num_workers=4)

In [8]:
# Checking the dataset
print("Training Set:\n")
for images, labels in train_loader:
    print("Image batch dimensions: ", images.shape)
    print("Image label dimensions: ", labels.shape)
    break

# Checking the dataset
print("\nValidation Set:")
for images, labels in valid_loader:
    print("Image batch dimensions: ", images.shape)
    print("Image label dimensions: ", labels.shape)
    break

# Checking the dataset
print("\nTest Set:")
for images, labels in test_loader:
    print("Image batch dimensions: ", images.shape)
    print("Image label dimensions: ", labels.shape)
    break

Training Set:

Image batch dimensions:  torch.Size([256, 3, 64, 64])
Image label dimensions:  torch.Size([256])

Validation Set:
Image batch dimensions:  torch.Size([256, 3, 64, 64])
Image label dimensions:  torch.Size([256])

Test Set:
Image batch dimensions:  torch.Size([256, 3, 64, 64])
Image label dimensions:  torch.Size([256])


In [11]:
torch.manual_seed(random_seed)
model=AlexNet(num_classes=num_classes).to(device)

optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
# Training
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples=0, 0
    model.eval()
    for i, (features, targets) in enumerate(data_loader):
        
        features=features.to(device)
        targets=targets.to(device)
        
        logits, probas=model(features)
        _, predicted_labels=torch.max(probas, 1)
        num_examples+=targets.size(0)
        assert predicted_labels.size()==targets.size()
        correct_pred+=(predicted_labels==targets).sum()
    return correct_pred.float()/num_examples*100


start_time=time.time()
cost_list=[]
train_acc_list, valid_acc_list=[], []

for epoch in range(num_epochs):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features=features.to(device)
        targets=targets.to(device)
        
        # Forward and backprop
        logits, probas=model(features)
        cost=F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        # Update model parameters
        optimizer.step()
        
        # ONLY FOR LOGGING
        cost_list.append(cost.item())
        if not batch_idx % 150:
            print("Batch: %03d/%03d | Epoch: %03d/%03d | Cost: %.2f" % (batch_idx, len(train_loader), epoch+1, num_epochs, cost))
        
    model.eval()
    with torch.set_grad_enabled(False):

        train_acc=compute_accuracy(model, train_loader, device)
        valid_acc=compute_accuracy(model, valid_loader, device)

        print(f'Epoch: {epoch+1:03d}/{num_epochs:03d}\n'
              f'Train ACC: {train_acc:.2f} | Validation ACC: {valid_acc:.2f}')
        
        train_acc_list.append(train_acc)
        valid_acc_list.append(valid_acc)
    
    elapsed=(time.time()-start_time)/60
    print(f'Time Elapsed: {elapsed:.2f} min')

elapsed=(time.time()-start_time)/60
print(f'Total Training Time: {elapsed:.2f} min')

Batch: 000/188 | Epoch: 001/020 | Cost: 2.30
Batch: 150/188 | Epoch: 001/020 | Cost: 1.77
Epoch: 001/020
Train ACC: 32.51 | Validation ACC: 33.35
Time Elapsed: 0.54 min
Batch: 000/188 | Epoch: 002/020 | Cost: 1.71
Batch: 150/188 | Epoch: 002/020 | Cost: 1.53
Epoch: 002/020
Train ACC: 43.76 | Validation ACC: 43.00
Time Elapsed: 1.07 min
Batch: 000/188 | Epoch: 003/020 | Cost: 1.46
Batch: 150/188 | Epoch: 003/020 | Cost: 1.31
Epoch: 003/020
Train ACC: 50.90 | Validation ACC: 51.90
Time Elapsed: 1.61 min
Batch: 000/188 | Epoch: 004/020 | Cost: 1.25
Batch: 150/188 | Epoch: 004/020 | Cost: 1.22
Epoch: 004/020
Train ACC: 53.08 | Validation ACC: 53.55
Time Elapsed: 2.15 min
Batch: 000/188 | Epoch: 005/020 | Cost: 1.16
Batch: 150/188 | Epoch: 005/020 | Cost: 1.14
Epoch: 005/020
Train ACC: 56.93 | Validation ACC: 57.65
Time Elapsed: 2.69 min
Batch: 000/188 | Epoch: 006/020 | Cost: 1.07
Batch: 150/188 | Epoch: 006/020 | Cost: 1.04
Epoch: 006/020
Train ACC: 61.80 | Validation ACC: 61.20
Time Elap

In [14]:
model.eval()
with torch.set_grad_enabled(False): # save memory during inference
    test_acc=compute_accuracy(model=model, data_loader=test_loader, device=device)
    valid_acc=compute_accuracy(model=model, data_loader=valid_loader, device=device)

print("Valid Accuracy: %.2f%% || Test Accuracy: %.2f%%" % (valid_acc, test_acc))

Valid Accuracy: 69.80% || Test Accuracy: 70.18%
